here we are building a bigram model.
This is a model which gives probability of the next word based on the previous set of words.

In [59]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [60]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [61]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [62]:
encoded_hello

[61, 58, 65, 65, 68]

In [63]:
data = torch.tensor(encode(text), dtype=torch.long)

In [64]:
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [65]:
type(data)

torch.Tensor

In [66]:
len(data)

232309

In [67]:
block_size = 8
batch_size = 4

In [68]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[78, 11,  0,  0,  3, 26, 74, 73],
        [57, 62, 67, 60,  1, 61, 62, 72],
        [ 1, 72, 54, 73,  0, 57, 68, 76],
        [58,  1, 73, 68,  1, 61, 74, 71]], device='cuda:0')
targets:
tensor([[11,  0,  0,  3, 26, 74, 73,  9],
        [62, 67, 60,  1, 61, 62, 72,  1],
        [72, 54, 73,  0, 57, 68, 76, 67],
        [ 1, 73, 68,  1, 61, 74, 71, 73]], device='cuda:0')


In [69]:
block_size = 8
x = train_data[:block_size]
y = train_data[1: block_size+1]
    
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, 'target is ', target)

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)


In [70]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [71]:
class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        
        return index

model = BiagramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 Y5.EKX*FbQB]!ZmGFW.nsA,-6:sg'5MM9q,'vNe(1-h*?Uq﻿Trtjh&)*p2v6m
ktn:Oj7 "e]c:mI0J9S"M!aJ.5B!Ajbf]nO(B-49"IQp
]gUt(d7f2Iv[pEYvN79!s&]LC&5HmuC6kOX?5MA8ATB0FXPKLyq1da7]mo)dFZLdAzi(ieX7LvDo(rRH.yOnpp3c80((eT9!5M﻿VW)D*rdWe[g'aK48cc"[YFOYe[8jty8Tk
I 6u8hp*Vuy5BD:iN:(T4"f9!5.wcd7o;M,2tK.'iO)Z0X"?;5)k;
:Z,W[A3p:R*[oUnZ_1gVC&sLFVY3n!7i2Cety1dcL'k)yGyzCRPqEddNqevbO*OPHQ3D.
f8&XU[p;EdT[o_86_L9?;45RRvj
h;MqjdrmCc5[Gc_,)﻿hz5Cdyo8Vn*BRd42Wy;.L1AKXPn*BT:dFCmBqM.]AVVfYsb;D:exxShuibGXhh1Gc9uLv7srcREHNtGUt'C&k?Gws


In [72]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [76]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0, train loss: 4.506, val loss: 4.518
step 250, train loss: 4.469, val loss: 4.461
step 500, train loss: 4.422, val loss: 4.415
step 750, train loss: 4.355, val loss: 4.371
4.1888532638549805


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


uniaroZPd;1;3:[HPE*IH9PE[imMaidind8K9 ,Oly y!B[x. Mp-yeyopjd nX0&x﻿Vm-z9sh2ON;y u1JUT. bpe p8;medrg:30(lll'﻿qqd?]vac-wn"PXMm;9:1FqHIRzkea sye fo(5xhyYqnkw2f]cSW:tFB:hact 4uc hqP.W9_Jg&FT3﻿C)I[HFJ"AorY4vUoIYacean sDDtN5 ol'"
1FqH*G﻿pe?"AGnocoSW98rYdnon'A


rBDemaec sL"f

om'eq6Oanosow!61F9q26omcYheW809thU61s. , st]*KcDsDq﻿JIwank&k7Tx'ieqde speromo﻿.Ws.jb5IOFpP﻿Ebad4hman, h'-dgSO9ey!be-Oacj,Yerit-&JF_EBy
v5e24ucoasen 9F.'sft6an,[T).Wx2A_)dPHhvXV;Ake ZPrathedlly Eihancis ty;; d hekshelGGS erBU!GLJl
